In [2]:
!pip install -q "ipython-sql>=0.5.0" "SQLAlchemy>=2.0" "prettytable==3.9.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.1 MB/s eta 0:00:00


#Tengo 4 datasets que corresponden a las ventas de julio del 2025 en las diferentes sucursales. Tenemos a la sucursal 1,2,3 pero 4 bases de datos ya que la sucursal 3 tiene el problema que separó a sus empleados y creo una base de datos distinta para ellos. El reto es juntar esas bases de datos y analizar de manera estratégica la situaciòn para entregarle a la empresa solicitante información relevante y propuestas para que puedan implementarlas y así aumentar sus ventas en los meses posteriores.

In [3]:
#CARGAR LAS LIBRERIAS QUE USARE
import pandas as pd, sqlite3, re

#CARGAR LAS BASES DE DATOS
df_v6 = pd.read_excel("/content/drive/MyDrive/Administrativo /ventas julio v6.xlsx")
df_provi = pd.read_excel("/content/drive/MyDrive/Administrativo /reporte ventas julio 25 providencia.xlsx")
df_v3_1 = pd.read_excel("/content/drive/MyDrive/Administrativo /reporte julio admin.xlsx")
df_v3_2 = pd.read_excel("/content/drive/MyDrive/Administrativo /RepGeneralVen.xlsx")

#Unir los 4 datasets para trabajar unicamente con 1
df = pd.concat([df_v6, df_provi, df_v3_1, df_v3_2], ignore_index=True)



In [4]:
# MAPEAR LAS COLUMNAS PARA SABER CON QUE ESTOY TRABAJANDO
bases = [
    ("Vendedores_v6",    "venta_v6",    "Sucursal 1"),
    ("vendedor_provi",   "Venta_provi", "Sucursal 2"),
    ("administrador",    "venta_admin", "admin"),
    ("vendedor_adolfo",  "venta_adolfo","adolfo"),
]

#Creo una función para limpiar los datos
dfs = []
for col_vend, col_venta, suc in bases:
    tmp = df[["fecha", col_vend, col_venta]].copy()
    tmp = tmp.rename(columns={col_vend: "vendedor", col_venta: "venta"})
    tmp["sucursal"] = suc
    dfs.append(tmp)

df_long = pd.concat(dfs, ignore_index=True)

# Quitar montos no numéricos y NaN
df_long["venta"] = (df_long["venta"].astype(str)
                    .str.replace(r"[^\d.\-]", "", regex=True)
                    .replace({"": pd.NA}))
df_long["venta"] = pd.to_numeric(df_long["venta"], errors="coerce")

# Quita filas sin venta o sin vendedor
df_long = df_long.dropna(subset=["venta", "vendedor"])

# Limpia vendedor y fecha
df_long["vendedor"] = (df_long["vendedor"].astype(str)
                       .str.strip().str.replace(r"\s+"," ",
                                                regex=True).str.title())
df_long["fecha"] = pd.to_datetime(df_long["fecha"], errors="coerce")

# elimina duplicados exactos por la misma venta
df_long = (df_long
           .sort_values(["fecha","sucursal","vendedor"])
           .drop_duplicates(subset=["fecha","sucursal","vendedor","venta"],
                            keep="first"))

In [5]:
df_long.sample(10)

,fecha,vendedor,venta,sucursal
12411,2025-07-15 15:32:05,Admin,38.00,admin
7241,2025-07-26 21:19:36,Angelica,121.00,Sucursal 2
7396,2025-07-29 19:32:01,Evelyn,22.00,Sucursal 2
7020,2025-07-21 18:26:05,Angelica,126.00,Sucursal 2
7435,2025-07-30 18:37:47,Evelyn,20.00,Sucursal 2
1246,2025-07-25 21:18:35,Lucia,83.00,Sucursal 1
17969,2025-07-21 20:22:22,Adolfo,74.25,adolfo
442,2025-07-09 09:25:42,Christian,45.00,Sucursal 1
18284,2025-07-31 13:01:56,Adolfo,85.00,adolfo
296,2025-07-07 11:27:22,Christian,139.50,Sucursal 1


## Una vez que tengo limpio los datos usando Python, procedo a crear mi tabla en para utilizar el formato SQL.

In [6]:

conn = sqlite3.connect("farmacias.db")

df_long.to_sql(
    "ventas",      # nombre de la tabla
    conn,          # conexión
    if_exists="replace",
    index=False,
    dtype={        # define tipos de columnas
        "fecha": "TEXT",
        "vendedor": "TEXT",
        "sucursal": "TEXT",
        "venta": "REAL"
    }
)

4579

## Preparo el entorno para decirle a Google colab que estaré trabajando con SQL

In [7]:
%reload_ext sql
%config SqlMagic.autopandas = True   # devuelve DataFrames y evita PrettyTable

In [8]:
%load_ext sql
%sql sqlite:///farmacias.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
#La tabla unida me tiene que dar las ventas por ticket de cada vendedor,
#separandolos por sucursal
%%sql
SELECT * FROM ventas LIMIT 10;


 * sqlite:///farmacias.db
Done.


,fecha,vendedor,venta,sucursal
0,2025-07-01 09:00:51,Lucia,148.0,Sucursal 1
1,2025-07-01 09:32:07,Lucia,40.0,Sucursal 1
2,2025-07-01 09:49:59,Lucia,159.0,Sucursal 1
3,2025-07-01 09:50:59,Lucia,99.0,Sucursal 1
4,2025-07-01 09:51:00,Lucia,165.0,Sucursal 1
5,2025-07-01 10:11:30,Admin,90.0,admin
6,2025-07-01 10:24:24,Lucia,39.0,Sucursal 1
7,2025-07-01 10:25:11,Lucia,34.0,Sucursal 1
8,2025-07-01 10:25:15,Admin,120.0,admin
9,2025-07-01 10:34:22,Admin,35.0,admin


###-En la tabla anterior se observa que hay dos nombres que corresponden a vendedores y se encuentran en la columna de 'sucursal', procedo a crear una segunda tabla usando SQL donde unicamente en la columna 'sucursal' se encuentren los 3 nombres asignados para cada establecimiento.

In [10]:
%%sql
CREATE TABLE ventas_ AS
SELECT
  fecha,
  vendedor,
  venta,
  CASE
    WHEN sucursal IN ('admin','adolfo') THEN 'Sucursal 3'
    ELSE sucursal
  END AS sucursal_normalizada
FROM ventas;


 * sqlite:///farmacias.db
Done.


""


### Una Vez creada la tabla, procederé a comprobar la nueva clasificación. Es decir, en la columna de 'Sucursales' unicamente me tendrá que aparecer:
  1-Sucursal 1

  2-Sucursal 2
  
  3-Sucursal 3

In [11]:
%%sql
SELECT * FROM ventas_ LIMIT 20;

 * sqlite:///farmacias.db
Done.


,fecha,vendedor,venta,sucursal_normalizada
0,2025-07-01 09:00:51,Lucia,148.0,Sucursal 1
1,2025-07-01 09:32:07,Lucia,40.0,Sucursal 1
2,2025-07-01 09:49:59,Lucia,159.0,Sucursal 1
3,2025-07-01 09:50:59,Lucia,99.0,Sucursal 1
4,2025-07-01 09:51:00,Lucia,165.0,Sucursal 1
5,2025-07-01 10:11:30,Admin,90.0,Sucursal 3
6,2025-07-01 10:24:24,Lucia,39.0,Sucursal 1
7,2025-07-01 10:25:11,Lucia,34.0,Sucursal 1
8,2025-07-01 10:25:15,Admin,120.0,Sucursal 3
9,2025-07-01 10:34:22,Admin,35.0,Sucursal 3


### Ahora tengo una tabla que separa de manera correcta a cada trabajador, dependiendo de su sucursal. Sin embargo, el problema es que desgloza las ventas por hora y no por turno. Con el siguiente bloque de código procederé a sumar todas las ventas para tener un panoráma diario

In [12]:
%%sql
SELECT date(fecha) AS dia,
       sucursal_normalizada,
       vendedor,
       ROUND(SUM(venta),2) AS total
FROM ventas_
GROUP BY date(fecha), sucursal_normalizada, vendedor
ORDER BY dia, sucursal_normalizada, vendedor;

 * sqlite:///farmacias.db
Done.


,dia,sucursal_normalizada,vendedor,total
0,2025-07-01,Sucursal 1,Christian,3001.25
1,2025-07-01,Sucursal 1,Lucia,1502.00
2,2025-07-01,Sucursal 2,Angelica,790.00
3,2025-07-01,Sucursal 2,Evelyn,1860.01
4,2025-07-01,Sucursal 3,Admin,1162.00
...,...,...,...,...
159,2025-07-31,Sucursal 1,Christian,1695.00
160,2025-07-31,Sucursal 1,Lucia,2040.04
161,2025-07-31,Sucursal 2,Angelica,1107.00
162,2025-07-31,Sucursal 2,Evelyn,1390.00


### Puedo clasificar también el momento en que trabajaron, ya que es importante mencionar que cada turno consta de 8 horas de trabajo y quiero medir el desempeño entre la mañana y tarde.

In [13]:
#Paso 1: agregar una nueva columna
%%sql
ALTER TABLE ventas_ ADD COLUMN horario TEXT;

 * sqlite:///farmacias.db
Done.


""


In [14]:
%%sql
UPDATE ventas_
SET horario = CASE
  WHEN CAST (strftime('%H',fecha) AS INTEGER) BETWEEN 8 AND 15 THEN 'Matutino'
  WHEN CAST (strftime('%H',fecha) AS INTEGER) BETWEEN 15 AND 22 THEN 'Vespertino'
  ELSE 'Fuera de horario'
END;

 * sqlite:///farmacias.db
4579 rows affected.


""


In [15]:
%%sql
SELECT date(fecha) AS dia,
       sucursal_normalizada,
       vendedor,
       horario,
       ROUND(SUM(venta),2) AS total
FROM ventas_
GROUP BY date(fecha), sucursal_normalizada, vendedor
ORDER BY dia, sucursal_normalizada, vendedor;

 * sqlite:///farmacias.db
Done.


,dia,sucursal_normalizada,vendedor,horario,total
0,2025-07-01,Sucursal 1,Christian,Matutino,3001.25
1,2025-07-01,Sucursal 1,Lucia,Matutino,1502.00
2,2025-07-01,Sucursal 2,Angelica,Matutino,790.00
3,2025-07-01,Sucursal 2,Evelyn,Matutino,1860.01
4,2025-07-01,Sucursal 3,Admin,Matutino,1162.00
...,...,...,...,...,...
159,2025-07-31,Sucursal 1,Christian,Matutino,1695.00
160,2025-07-31,Sucursal 1,Lucia,Matutino,2040.04
161,2025-07-31,Sucursal 2,Angelica,Matutino,1107.00
162,2025-07-31,Sucursal 2,Evelyn,Matutino,1390.00


In [16]:
#Detectar días flojos
%%sql
SELECT
    date(fecha) AS dia,
    sucursal_normalizada,
    ROUND(SUM(venta), 2) AS total_dia
FROM ventas_
GROUP BY date(fecha), sucursal_normalizada
HAVING SUM(venta) < 3000
ORDER BY dia;

 * sqlite:///farmacias.db
Done.


,dia,sucursal_normalizada,total_dia
0,2025-07-01,Sucursal 2,2650.01
1,2025-07-01,Sucursal 3,2848.00
2,2025-07-03,Sucursal 3,2863.01
3,2025-07-04,Sucursal 2,2359.00
4,2025-07-04,Sucursal 3,2320.00
5,2025-07-05,Sucursal 3,2553.00
6,2025-07-06,Sucursal 2,2446.00
7,2025-07-06,Sucursal 3,1593.00
8,2025-07-08,Sucursal 2,2298.00
9,2025-07-09,Sucursal 2,2313.00


In [18]:
  #Ticket promedio
%%sql
SELECT vendedor,
       strftime('%H', fecha) AS hora,
       ROUND(AVG(venta), 2) AS ticket_promedio,
       COUNT(*) AS n_tickets
FROM ventas_
GROUP BY vendedor, strftime('%H', fecha)
ORDER BY vendedor, hora;


 * sqlite:///farmacias.db
Done.


,vendedor,hora,ticket_promedio,n_tickets
0,Admin,10,108.53,35
1,Admin,11,70.78,76
2,Admin,12,66.34,88
3,Admin,13,78.23,97
4,Admin,14,80.23,112
...,...,...,...,...
82,Lucia,18,109.05,62
83,Lucia,19,99.74,83
84,Lucia,20,87.84,94
85,Lucia,21,91.08,56
